In [1]:
from dolfinx.io import XDMFFile
from dolfinx.mesh import meshtags_from_entities
from dolfinx.cpp.mesh import cell_entity_type
from dolfinx.io import distribute_entity_data
from dolfinx.graph import adjacencylist
from dolfinx.mesh import create_mesh
from dolfinx.cpp.mesh import to_type
from dolfinx.cpp.io import perm_gmsh
import numpy
from dolfinx.io import gmshio
from mpi4py import MPI
from dolfinx.io.gmshio import model_to_mesh
import numpy as np
import gmsh
import warnings
warnings.filterwarnings("ignore")
gmsh.initialize()

The first step is to create the mesh that we will be performing the analysis on. In this case it will be a model of the silicon test masses for the Gingin 7m cavity project. These are 10cm in diameter and 3cm thick. They have a circular cross section with flat sections on two edges. 

To create the mesh we will be using the python API for gmsh. An open source software for creating finite element models. We have already initialised gmsh which must be done prior to using it. We then add the model "test_mass" to gmsh.

In [2]:
gmsh.model.add("test_mass")

In [3]:
cad = gmsh.model.geo
lc = 0.002


The variable lc will determine the mesh size. To create the model we begin by defing several points and then connecting them with lines. We then define a line loop which allows a surface to be created. We extrude this surface to create the volume.

In [4]:
cad.addPoint(0, -0.05, 0.01, lc, 1)
cad.addPoint(0, 0.05, 0.01, lc, 2)
cad.addPoint(0, 0.05, -0.01, lc, 3)
cad.addPoint(0, -0.05, -0.01, lc, 4)
cad.addPoint(0,0,0, lc, 5)
cad.addLine(2, 3, 1)
cad.addLine(4, 1, 2)
cad.addCircleArc(1, 5, 2, 3)
cad.addCircleArc(3, 5, 4, 4)
cad.addCurveLoop([3, 1, 4, 2 ], 10)
cad.addPlaneSurface([10], 11)

11

In [5]:
vol = cad.extrude([(2, 11)], 0.03,0,0 )

We must then synchronise the model so that the various pieces are combined

In [6]:
cad.synchronize()

In [7]:
print(vol)

[(2, 33), (3, 1), (2, 20), (2, 24), (2, 28), (2, 32)]


We then have to tell gmsh which elements are physical groups. We define two physical groups one for the volume and another for the surface of the model.

In [8]:
gmsh.model.addPhysicalGroup(3, [vol[1][1]], 41, "volume")
gmsh.model.addPhysicalGroup(2, [vol[0][1],vol[2][1], vol[3][1], vol[4][1], vol[5][1]], 42, "surface")





42

Now we are able to generate the mesh

In [9]:
gmsh.model.mesh.generate(3)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Circle)
Info    : [ 30%] Meshing curve 4 (Circle)
Info    : [ 40%] Meshing curve 13 (Circle)
Info    : [ 50%] Meshing curve 14 (Line)
Info    : [ 50%] Meshing curve 15 (Circle)
Info    : [ 60%] Meshing curve 16 (Line)
Info    : [ 70%] Meshing curve 18 (Line)
Info    : [ 80%] Meshing curve 19 (Line)
Info    : [ 90%] Meshing curve 23 (Line)
Info    : [100%] Meshing curve 27 (Line)
Info    : Done meshing 1D (Wall 0.0017634s, CPU 0.00303s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 11 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 20 (Surface, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 24 (Surface, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 28 (Surface, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 32 (Surface, Frontal-Delaunay)
Info    : [ 90%] Meshing surface 33 (Plane, Frontal-Delaunay)
Info    : Done mes

In [10]:
gmsh.write('mirror2.msh')

Info    : Writing 'mirror2.msh'...


Info    : Done writing 'mirror2.msh'


The gmsh model can now be imported into dolfinx

In [11]:
model_rank = 0
domain, cell_tags, facet_tags = model_to_mesh(
    gmsh.model, MPI.COMM_WORLD, model_rank,3)

In [12]:
import pyvista
from dolfinx import mesh, fem, plot, io, default_scalar_type
from dolfinx.fem.petsc import LinearProblem
from mpi4py import MPI
import ufl
import numpy as np
E = 130e9
nu = 0.27
mu = E / (2.0 * (1.0 + nu))
lambda_ = E * nu / ((1.0 + nu) * (1.0 - 2.0 * nu))
rho = 2329
g = 9.8

We now begin to construct the model for the finite element analysis

In [13]:
V = fem.VectorFunctionSpace(domain, ("Lagrange", 1))
    

In [33]:
def clamped_boundary(x):
    return np.logical_or(np.isclose(x[1], -0.05), np.isclose(x[1], 0.05))


fdim = domain.topology.dim - 1
boundary_facets = mesh.locate_entities_boundary(domain, fdim, clamped_boundary)



In [34]:
u_D = np.array([0, 0, 0], dtype=default_scalar_type)


In [35]:
bc = fem.dirichletbc(u_D, fem.locate_dofs_topological(V, fdim, boundary_facets), V)

In [36]:
T = fem.Constant(domain, default_scalar_type((0, 0, 0)))

In [37]:
ds = ufl.Measure("ds", domain=domain)

In [38]:
def epsilon(u):
    return ufl.sym(ufl.grad(u))  # Equivalent to 0.5*(ufl.nabla_grad(u) + ufl.nabla_grad(u).T)


def sigma(u):
    return lambda_ * ufl.nabla_div(u) * ufl.Identity(len(u)) + 2 * mu * epsilon(u)


u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
f = fem.Constant(domain, default_scalar_type((0, 0, -rho * g)))
a = ufl.inner(sigma(u), epsilon(v)) * ufl.dx
L = ufl.dot(f, v) * ufl.dx + ufl.dot(T, v) * ds

In [39]:
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

The solver will return the solution with the displacement this can then be plotted in pyvista

In [40]:
pyvista.start_xvfb()

# Create plotter and pyvista grid
p = pyvista.Plotter()
topology, cell_types, geometry = plot.vtk_mesh(V)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

# Attach vector values to grid and warp grid by vector
grid["u"] = uh.x.array.reshape((geometry.shape[0], 3))
#actor_0 = p.add_mesh(grid, style="wireframe", color="k")
warped = grid.warp_by_vector("u", factor=1.5)
actor_1 = p.add_mesh(warped, show_edges=False)
p.show_axes()
if not pyvista.OFF_SCREEN:
    p.show()
else:
    figure_as_array = p.screenshot("deflection.png")

Widget(value="<iframe src='http://localhost:46817/index.html?ui=P_0x7fa9880c4250_4&reconnect=auto' style='widt…

In [41]:
with io.XDMFFile(domain.comm, "deformation.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    uh.name = "Deformation"
    xdmf.write_function(uh)

Now we want to convert from the deformation to the stress tensor. In order to do this we must create a new vector space and interpolate onto it with our expression for the stress tensor

In [42]:
gdim = domain.geometry.dim
Stress = fem.VectorFunctionSpace(domain, ("Discontinuous Lagrange", 0, (gdim,)), 9)
stress = fem.Function(Stress)
stress_expr = fem.Expression(sigma(uh), Stress.element.interpolation_points())

In [43]:
stress.interpolate(stress_expr)

In [44]:
stress_array = stress.x.array.reshape((int(stress.x.array.shape[0]/9), 3, 3))

Now we just define our functions for calculating the birefringence and find the birefringence of each element

In [45]:
def rot_mat(rot_z, rot_y, rot_x):
    a = rot_z
    b = rot_y
    y = rot_x
    mat = np.array([[np.cos(a)*np.cos(b), np.cos(a)*np.sin(b)*np.sin(y) - np.sin(a)*np.cos(y), np.cos(a)*np.sin(b)*np.cos(y) + np.sin(a)*np.sin(y)],[np.sin(a)*np.cos(b), np.sin(a)*np.sin(b)*np.sin(y) + np.cos(a)*np.cos(y), np.sin(a)*np.sin(b)*np.cos(y) - np.cos(a)*np.sin(y)], [-1*np.sin(b), np.cos(b)*np.sin(y), np.cos(b)*np.cos(y)]])
    return mat


def h_eigenvalues(mat):
    a = mat[0, 0]
    b = mat[1, 1]
    c = mat[0, 1]

    l1 = (a + b - np.sqrt(4 * c ** 2 + (a - b) ** 2)) / 2
    l2 = (a + b + np.sqrt(4 * c ** 2 + (a - b) ** 2)) / 2
    return l1, l2


def r_index(mat):
    l1, l2 = h_eigenvalues(mat)
    return (1/np.sqrt(l1)) - (1/np.sqrt(l2))


def stress(s1, s2, s3, s4, s5, s6):
    arr = np.array([s1, s2, s3, s4, s5, s6])
    return arr


def photoelastic(p11, p12, p44):
    arr = np.array([[p11, p12, p12, 0, 0, 0], [p12, p11, p12, 0,0,0], [p12, p12, p11, 0,0,0],[0,0,0,p44,0,0],[0,0,0,0,p44,0],[0,0,0,0,0,p44]])
    return arr


def elastic(PR, YM):
    arr = np.array([[1, -1*PR, -1*PR,0,0,0],[-1*PR, 1, -1*PR, 0,0,0], [-1*PR, -1*PR, 1,0,0,0],[0,0,0,1+PR,0,0],[0,0,0,0,1+PR,0],[0,0,0,0,0,1+PR]])
    arr = arr * (1/YM)
    return arr


def delta_b(PR, YM, p11, p12, p44, s1, s2, s3, s4, s5, s6):
    arr = np.dot(photoelastic(p11, p12, p44), np.dot(elastic(PR, YM), stress(s1, s2, s3, s4, s5, s6)))
    return arr


def b_0(nx, ny, nz):
    arr = np.array([1/(nx**2), 1/(ny**2), 1/(nz**2), 0,0,0])
    return arr


def b_new(PR, YM, s1, s2, s3, s4, s5, s6, p11, p12, p44, nx, ny, nz):
    delB = delta_b(PR, YM, p11, p12, p44, s1, s2, s3, s4, s5, s6)
    B0 = b_0(nx, ny, nz)
    B = delB + B0
    arr2 = np.array([[B[0], B[5], B[4]], [B[5], B[1], B[3]], [B[4], B[3], B[2]]])
    return arr2


def birefringence(rot_z, rot_x, rot_y, PR, YM, s1, s2, s3, s4, s5, s6, p11, p12, p44, nx, ny, nz):
    b = b_new(PR, YM, s1, s2, s3, s4, s5, s6, p11, p12, p44, nx, ny, nz)
    rot = rot_mat(rot_z, rot_x, rot_y)
    rotated = np.dot(rot, np.dot(b, np.linalg.inv(rot)))
    small_b = rotated[1:, 1:]
    delta_n = r_index(small_b)
    return delta_n


In [46]:
npoints = stress_array.shape[0]
dn = np.zeros(npoints)
for x in range(npoints):
    a = stress_array[x]
    dn[x] = birefringence(rot_z=0, rot_x=0, rot_y=0, PR=0.27, YM=130e9, s1=a[0,0], s2=a[1,1], s3=a[2,2],s4= a[1,2], s5=a[0,2],s6=a[0,1], p11=-0.094, p12=0.017,p44= -0.051, nx=3.48, ny=3.48, nz=3.48)


Now we can plot the birefringence and we can observe that this is very close to what was observed

In [64]:
pyvista.global_theme.cmap = 'jet'
warped.cell_data["Birefringence"] = dn
warped.set_active_scalars("Birefringence")
p = pyvista.Plotter()
p.add_mesh(warped, clim=[0, 1e-7])
p.show_axes()
if not pyvista.OFF_SCREEN:
    p.show()
else:
    stress_figure = p.screenshot(f"stresses.png")

Widget(value="<iframe src='http://localhost:46817/index.html?ui=P_0x7fa9881012d0_18&reconnect=auto' style='wid…

In [48]:
dn2 = np.zeros(npoints)
for x in range(npoints):
    a = stress_array[x]
    dn2[x] = birefringence(rot_z=0.2, rot_x=0, rot_y=0, PR=0.27, YM=130e9, s1=a[0,0], s2=a[1,1], s3=a[2,2],s4= a[1,2], s5=a[0,2],s6=a[0,1], p11=-0.094, p12=0.017,p44= -0.051, nx=3.48, ny=3.48, nz=3.48)


In [58]:
warped.cell_data["BirefringenceBS"] = dn2
warped.set_active_scalars("BirefringenceBS")
p = pyvista.Plotter()
p.add_mesh(warped, clim=[0, 1e-7])
p.show_axes()
if not pyvista.OFF_SCREEN:
    p.show()
else:
    stress_figure = p.screenshot(f"stresses_BS.png")

Widget(value="<iframe src='http://localhost:46817/index.html?ui=P_0x7fa98ac9ead0_15&reconnect=auto' style='wid…

In [66]:
warped.cell_data["BirefringenceDiff"] = dn2-dn
warped.set_active_scalars("BirefringenceDiff")
p = pyvista.Plotter()
p.add_mesh(warped, clim=[-1e-9, 0])
p.show_axes()
if not pyvista.OFF_SCREEN:
    p.show()
else:
    stress_figure = p.screenshot(f"stresses_Diff.png")

Widget(value="<iframe src='http://localhost:46817/index.html?ui=P_0x7fa9880fc310_20&reconnect=auto' style='wid…